# An fMRI study of composition in noun and verb phrases
### Bonnasse-Gahot, Bemis, Perez-Guevara, Dehaene, Pallier
Compute contrasts for all subjects, and group, for the localizer.

In [1]:
import numpy as np
import os.path as op
from joblib import load
import nibabel as nib

import simpcomp as sc

output_path = op.join(sc.localizer_folder, 'contrasts')
sc.make_dir(output_path)

In [2]:
for sub_id in sc.subject_list:
    sub_glm_filename = op.join(sc.localizer_folder, 'glm', 'glm_{}'.format(sub_id))
    fmri_glm = load(sub_glm_filename)

    design_matrix = fmri_glm.design_matrices_[0]
    contrast_matrix = np.eye(design_matrix.shape[1])
    contrasts_basic = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(design_matrix.columns)])

    contrasts = {}
    contrasts['sentence_pseudo'] = contrasts_basic['sentence'] - contrasts_basic['pseudo']

    for name, val in contrasts.items():
        z_map = fmri_glm.compute_contrast(val, output_type='z_score')
        t_map = fmri_glm.compute_contrast(val, output_type='stat')
        eff_map = fmri_glm.compute_contrast(val, output_type='effect_size')
        nib.save(z_map, op.join(output_path, '{}_{}_zmap.nii.gz'.format(name, sub_id)))
        nib.save(t_map, op.join(output_path,'{}_{}_tmap.nii.gz'.format(name, sub_id)))
        nib.save(eff_map, op.join(output_path, '{}_{}_effsize.nii.gz'.format(name, sub_id)))

In [3]:
# group analysis
for contrast in contrasts:
    maps = [op.join(output_path, '{}_{}_effsize.nii.gz'.format(contrast,sub_tmp)) for sub_tmp in sc.subject_list]   
    if maps == []:
        print("%s : no file with this mask" % mask)
    else:
        sc.create_one_sample_t_test(contrast, maps, output_path)  